Paso 0 — Reset seguro de la colección (opcional)

In [1]:
# --- Paso 0: Reset opcional de la colección ---

import os, chromadb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

VDB_PATH = "./vectordb"
COLLECTION_NAME = "autoselx_docs"

def reset_collection(full_reset=False, collection_name=COLLECTION_NAME):
    import chromadb
    if full_reset:
        if os.path.exists(VDB_PATH):
            shutil.rmtree(VDB_PATH)
            print(f"🧹 Directorio {VDB_PATH} borrado por completo.")
        os.makedirs(VDB_PATH, exist_ok=True)
    else:
        client = chromadb.PersistentClient(path=VDB_PATH)
        try:
            client.delete_collection(name=collection_name)
            print(f"🧹 Colección '{collection_name}' eliminada.")
        except Exception as e:
            print(f"(info) No se eliminó colección: {e}")



In [ ]:
# --- Paso 1: imports y utilidades ---
import os, re, pdfplumber
from typing import List, Tuple, Dict
from collections import Counter

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document   # ✅ Import correcto

from dotenv import load_dotenv
import os

# Cargar variables del archivo .env
load_dotenv()

# Leer la API key desde el entorno
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# --- Patrones de limpieza (encabezados/pies repetidos) ---
CLEAN_PATTERNS = [
    r"FORMATO REQUISICIÓN.*",
    r"ECOPETROL DESARROLLO DE PROYECTO.*",
    r"Todos los derechos reservados.*",
    r"EDP-F-046.*Versión.*",
    r"MR.*#:.*CAS.*",
    r"Página\s+\d+\s+de\s+\d+",
    r"_{6,}",  # líneas largas
]

def clean_text(text: str) -> str:
    for p in CLEAN_PATTERNS:
        text = re.sub(p, "", text, flags=re.IGNORECASE)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

# --- Palabras clave para asignar prioridad ---
HD_HINTS = [
    r"\bAPI\s*675\b", r"HOJA\s*DE\s*DATOS", r"\bDATA\s*SHEET\b", r"\bDATASHEET\b",
    r"BOMB(A|S)?\s+(DOSIFICADORA|DOSING)"
]

ALCANCE_HINTS = [
    r"\bALCANCE\b", r"ALCANCE\s+DEL\s+SUMINISTRO", r"\bSCOPE\b", r"SCOPE\s+OF\s+SUPPLY"
]

REQ_HINTS = [
    r"\bREQUISITO(S)?\b", r"\bREQUERIMIENTO(S)?\b",
    r"REQUISITO(S)?\s+T(É|E)CNICO(S)?", r"CONDICIONES?\s+T(É|E)CNICAS?",
    r"CRITERIOS?\s+DE\s+DISE(Ñ|N)O", r"DATOS?\s+DE\s+DISE(Ñ|N)O",
    r"\bREQUIREMENT(S)?\b", r"DESIGN\s+REQUIREMENT(S)?\b",
    r"TECHNICAL\s+REQUIREMENT(S)?\b", r"\bSPECIFICATION(S)?\b",
    r"MINIMUM\s+REQUIREMENT(S)?\b",
    r"\bEL\s+PROVEEDOR\s+DEBE\b", r"\bEL\s+VENDEDOR\s+DEBE\b",
    r"\bDEBERÁ\b", r"\bSE\s+REQUIERE(N)?\b",
    r"\bSHALL\b", r"\bMUST\b", r"\bIS\s+REQUIRED\s+TO\b"
]

NORM_HINTS = [
    r"\bNORMAS?\b", r"\bEST[ÁA]NDA(RE|R)E?S?\b",
    r"\bC(Ó|O)DIGO(S)?\b", r"\bCODE(S)?\b", r"\bSTANDARD(S)?\b",
    r"\bSPEC(S|IFICATIONS)?\b", r"\bREGULATION(S)?\b",
    r"\bRETIE\b", r"\bAPI\b", r"\bASTM\b", r"\bASME\b", r"\bIEC\b", r"\bIEEE\b",
    r"APPLICABLE\s+(CODES|STANDARDS|SPECIFICATIONS)"
]

def matches_any(text: str, patterns: List[str]) -> bool:
    return any(re.search(p, text, flags=re.IGNORECASE) for p in patterns)

def detect_priority(text: str) -> str:
    """Clasifica el chunk en una categoría fija de prioridad"""
    if matches_any(text, HD_HINTS):      return "hoja_datos"
    if matches_any(text, ALCANCE_HINTS): return "alcance"
    if matches_any(text, REQ_HINTS):     return "requisito"
    if matches_any(text, NORM_HINTS):    return "norma"
    return "general"

# --- Conversión de tablas ---
def table_to_text_and_kv(rows: List[List[str]]) -> Tuple[str, Dict[str, str] | None]:
    clean_rows = []
    for r in rows:
        if not r:
            continue
        rr = [(c or "").strip() for c in r]
        if any(rr):
            clean_rows.append(rr)

    kv = None
    if clean_rows and max(len(r) for r in clean_rows) <= 2 and len(clean_rows) >= 2:
        kv = {}
        for r in clean_rows:
            if len(r) >= 2 and r[0] and r[1]:
                kv[r[0]] = r[1]

    text = "\n".join(" | ".join(r) for r in clean_rows)
    return text, kv if kv else None

# --- KV extractor unificado (texto + tablas) ---
def kv_from_text_page(text: str) -> Dict[str, str]:
    """
    Extrae pares clave-valor típicos de Hoja de Datos (texto o tabla).
    Soporta caudal, presión, potencia, viscosidad, redundancia, materiales, API 675.
    """
    t = " ".join(text.split()).upper()
    kv = {}

    # Caudal (GPH o LPH, con rangos)
    m = re.search(r"(\d+(?:[.,]\d+)?)\s*(?:-|–|A|TO)?\s*(\d+(?:[.,]\d+)?)?\s*GPH", t)
    if m: kv["caudal_gph"] = f"{m.group(1)}-{m.group(2)}" if m.group(2) else m.group(1)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*(?:-|–|A|TO)?\s*(\d+(?:[.,]\d+)?)?\s*LPH", t)
    if m: kv["caudal_lph"] = f"{m.group(1)}-{m.group(2)}" if m.group(2) else m.group(1)

    # Presión (PSI o BAR, con rangos)
    m = re.search(r"(\d+(?:[.,]\d+)?)\s*(?:-|–|A|TO)?\s*(\d+(?:[.,]\d+)?)?\s*PSI", t)
    if m: kv["presion_psig"] = f"{m.group(1)}-{m.group(2)}" if m.group(2) else m.group(1)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*(?:-|–|A|TO)?\s*(\d+(?:[.,]\d+)?)?\s*BAR", t)
    if m: kv["presion_bar"] = f"{m.group(1)}-{m.group(2)}" if m.group(2) else m.group(1)

    # Potencia (HP o KW)
    m = re.search(r"(\d+(?:[.,]\d+)?)\s*HP", t)
    if m: kv["hp_motor"] = m.group(1)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*KW", t)
    if m: kv["kw_motor"] = m.group(1)

    # Viscosidad (cps)
    m = re.search(r"(\d+)\s*-\s*(\d+)\s*CPS", t)
    if m:
        kv["rango_cps_min"] = m.group(1)
        kv["rango_cps_max"] = m.group(2)

    # Bombas operativas + respaldo (ej. 1+1)
    m = re.search(r"\b(\d+)\s*\+\s*(\d+)\b", t)
    if m:
        kv["bombas_operativas"] = m.group(1)
        kv["bombas_respaldo"] = m.group(2)

    # Materiales
    if "304" in t: kv["material_tanque"] = "304SS"
    if "316" in t: kv["material_mojado"] = "316SS"

    # Norma API 675
    if "API 675" in t: kv["api_675"] = "true"

    return kv



In [3]:
'''
# --- Paso 2: extracción de contenido del PDF (mejorado) ---
import json
import pdfplumber
from typing import List
from langchain_core.documents import Document  # ✅ Import correcto

def _is_text_clean(text: str, min_ratio: float = 0.7) -> bool:
    """Chequea que el texto tenga mayoría de caracteres legibles (no ruido PDF)."""
    if not text:
        return False
    clean_chars = sum(c.isalnum() or c.isspace() or c in ".,;:()-/%" for c in text)
    ratio = clean_chars / len(text)
    return ratio >= min_ratio

def extract_pdf_content(filepath: str) -> List[Document]:
    """
    Extrae texto y tablas del PDF con metadatos enriquecidos:
    - type: text/table
    - prio: hoja_datos, alcance, requisito, norma, general
    - kv: si se detecta información técnica
    """
    docs: List[Document] = []

    with pdfplumber.open(filepath) as pdf:
        for i, page in enumerate(pdf.pages, start=1):

            # --- 1) Texto plano ---
            raw_text = page.extract_text() or ""
            clean = clean_text(raw_text)

            if clean and _is_text_clean(clean):
                prio = detect_priority(clean)

                # ⚖️ Ajuste: si contiene "ALCANCE", fuerza prioridad a alcance
                if "ALCANCE" in clean.upper():
                    prio = "alcance"

                md = {"source": filepath, "page": i, "type": "text", "prio": prio}

                # 🔑 Extraer KV desde texto
                kv_text = kv_from_text_page(clean)
                if kv_text:
                    md["kv"] = json.dumps(kv_text, ensure_ascii=False)

                docs.append(Document(page_content=clean, metadata=md))

            # --- 2) Tablas ---
            try:
                tables = page.extract_tables() or []
            except Exception:
                tables = []

            for trows in tables:
                # Convertimos tabla en texto legible y posible dict KV directo
                table_text, kv_table = table_to_text_and_kv(trows)
                table_text = clean_text(table_text)

                if not table_text or not _is_text_clean(table_text):
                    continue

                prio = detect_priority(table_text)

                # ⚖️ Ajuste igual que arriba
                if "ALCANCE" in table_text.upper():
                    prio = "alcance"

                md = {"source": filepath, "page": i, "type": "table", "prio": prio}

                # 🔑 Detectar KV desde texto y tabla
                kv_detected = kv_from_text_page(table_text)
                if kv_table:
                    kv_detected.update(kv_table)

                if kv_detected:
                    md["kv"] = json.dumps(kv_detected, ensure_ascii=False)

                docs.append(Document(page_content=table_text, metadata=md))

    return docs
'''

'\n# --- Paso 2: extracción de contenido del PDF (mejorado) ---\nimport json\nimport pdfplumber\nfrom typing import List\nfrom langchain_core.documents import Document  # ✅ Import correcto\n\ndef _is_text_clean(text: str, min_ratio: float = 0.7) -> bool:\n    """Chequea que el texto tenga mayoría de caracteres legibles (no ruido PDF)."""\n    if not text:\n        return False\n    clean_chars = sum(c.isalnum() or c.isspace() or c in ".,;:()-/%" for c in text)\n    ratio = clean_chars / len(text)\n    return ratio >= min_ratio\n\ndef extract_pdf_content(filepath: str) -> List[Document]:\n    """\n    Extrae texto y tablas del PDF con metadatos enriquecidos:\n    - type: text/table\n    - prio: hoja_datos, alcance, requisito, norma, general\n    - kv: si se detecta información técnica\n    """\n    docs: List[Document] = []\n\n    with pdfplumber.open(filepath) as pdf:\n        for i, page in enumerate(pdf.pages, start=1):\n\n            # --- 1) Texto plano ---\n            raw_text = 

In [4]:
# --- Paso 2: extracción de contenido del PDF (robusto a páginas con tablas) --- PRUEBAAAAAAAAAa
import json, re, pdfplumber
from typing import List
from langchain_core.documents import Document  # ✅

# --- Heurística: detectar texto "corrupto/gibberish" típico de tablas mal leídas ---
def is_gibberish_text(t: str) -> bool:
    if not t or not t.strip():
        return True
    s = " ".join(t.split())

    # 1) Muchos puntos seguidos (líneas de puntos / leaders)
    if re.search(r"\.{5,}", s):
        return True

    # 2) Secuencias largas de letras sueltas separadas por espacios
    if re.search(r"(?:\b\w\b\s+){6,}", s):
        return True

    # 3) Ratio de tokens de una sola letra elevado
    tokens = re.findall(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ]+", s)
    if tokens:
        single = sum(1 for tok in tokens if len(tok) == 1)
        if single / max(len(tokens), 1) > 0.35:
            return True

    return False

def extract_pdf_content(filepath: str) -> List[Document]:
    """
    Extrae texto y tablas con metadatos:
      - type: text | table
      - prio: hoja_datos | alcance | requisito | norma | general
      - kv: dict (serializado) cuando se detecta info técnica
    Reglas:
      • Si la página tiene tablas y el texto plano luce "gibberish", no indexar el texto.
      • Para tablas: convertimos a texto legible y combinamos KV por estructura y por regex.
    """
    docs: List[Document] = []

    with pdfplumber.open(filepath) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            # --- 1) Tablas primero (sabemos si hay tablas para decidir sobre texto) ---
            try:
                tables = page.extract_tables() or []
            except Exception:
                tables = []
            has_tables = bool(tables)

            table_docs_this_page: List[Document] = []
            for trows in tables:
                table_text, kv_table = table_to_text_and_kv(trows)
                table_text = clean_text(table_text)
                if not table_text:
                    continue

                prio = detect_priority(table_text)
                md = {"source": filepath, "page": i, "type": "table", "prio": prio}

                # KV por regex + por estructura de 2 columnas
                kv_detected = kv_from_text_page(table_text)
                if kv_table:
                    kv_detected = {**kv_detected, **kv_table}
                if kv_detected:
                    md["kv"] = json.dumps(kv_detected, ensure_ascii=False)

                table_docs_this_page.append(Document(page_content=table_text, metadata=md))

            # --- 2) Texto plano (solo si vale la pena) ---
            raw_text = page.extract_text() or ""
            clean = clean_text(raw_text)
            text_looks_bad = is_gibberish_text(clean)

            # Si hay tablas y el texto luce corrupto, NO añadimos el texto de la página
            add_text = bool(clean) and not (has_tables and text_looks_bad)

            if add_text:
                prio = detect_priority(clean)
                md = {"source": filepath, "page": i, "type": "text", "prio": prio}

                kv_text = kv_from_text_page(clean)
                if kv_text:
                    md["kv"] = json.dumps(kv_text, ensure_ascii=False)

                docs.append(Document(page_content=clean, metadata=md))

            # --- 3) Añadir las tablas (siempre que tengan contenido) ---
            docs.extend(table_docs_this_page)

    return docs


In [5]:
# ============================================
# Paso 3 - Indexar en Chroma (persistente)
# ============================================
from collections import Counter
from langchain_chroma import Chroma   # ✅ usar langchain_chroma
from langchain_openai import OpenAIEmbeddings

# Configuración global
VDB_PATH = "./vectordb"
COLLECTION_NAME = "autoselx_docs"

def reset_collection():
    """Elimina la colección persistente (para recargar desde cero)."""
    import shutil, os
    if os.path.exists(VDB_PATH):
        shutil.rmtree(VDB_PATH)
        print(f"🗑️ Colección eliminada en {VDB_PATH}")

def _safe_persist(vs):
    """Compatibilidad de persistencia entre versiones de Chroma/langchain."""
    try:
        if hasattr(vs, "persist"):
            vs.persist()  # versiones antiguas
        elif hasattr(vs, "_client") and hasattr(vs._client, "persist"):
            vs._client.persist()  # versiones nuevas
    except Exception as e:
        print(f"(i) No se pudo forzar persistencia explícita: {e}")

def add_files_to_vectordb(filepath: str, reset: bool = False):
    """
    1) Extrae documentos del PDF (texto + tablas + KV)
    2) Aplica chunking conservando metadatos
    3) Indexa en Chroma persistente
    4) Muestra diagnóstico resumido
    """
    if reset:
        reset_collection()

    # --- 1) Extraer documentos brutos ---
    raw_docs = extract_pdf_content(filepath)

    # --- 2) Chunking ---
    splits = chunk_documents(raw_docs)

    # --- 3) Indexar en Chroma persistente ---
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=OpenAIEmbeddings(),
        persist_directory=VDB_PATH,
        collection_name=COLLECTION_NAME
    )

    # 🔑 Persistir en disco (seguro en múltiples versiones)
    _safe_persist(vectorstore)

    # --- 4) Diagnóstico ---
    by_type = Counter(d.metadata.get("type", "text") for d in splits)
    by_prio = Counter(d.metadata.get("prio", "general") for d in splits)
    total_kv = sum(1 for d in splits if "kv" in d.metadata)

    try:
        n_in_db = vectorstore._collection.count()
        print(f"📦 Documentos en DB (colección real): {n_in_db}")
    except Exception:
        print("(i) No se pudo obtener conteo directo desde DB")

    print(f"✅ {len(splits)} fragmentos indexados en {VDB_PATH} (colección '{COLLECTION_NAME}')")
    print("   📊 Por tipo:", dict(by_type))
    print("   📊 Por prioridad:", dict(by_prio))
    print(f"   🔑 Documentos con KV extraídos: {total_kv}")

    return vectorstore


In [6]:
# ============================================
# Paso 4 - Carga incremental y diagnóstico extendido
# ============================================
from collections import Counter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# === Utilidades de normalización y hashing  ===
import hashlib
import unicodedata
import re

def _normalize_for_hash(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    return s

def make_doc_id(meta: dict, content: str) -> str:
    """ID estable basado en (source|page|type|contenido normalizado)."""
    key = f"{meta.get('source','')}|{meta.get('page','')}|{meta.get('type','')}|{_normalize_for_hash(content)}"
    return hashlib.sha1(key.encode("utf-8")).hexdigest()

def dedupe_by_hash(docs: List[Document]) -> tuple[list[Document], list[str], int]:
    """Elimina duplicados por contenido normalizado. Devuelve (docs_unicos, ids, n_drops)."""
    seen = set()
    out_docs, ids = [], []
    drops = 0
    for d in docs:
        h = make_doc_id(d.metadata, d.page_content)
        if h in seen:
            drops += 1
            continue
        seen.add(h)
        # Guarda el hash en metadatos (útil para debug)
        d.metadata["content_hash"] = h
        out_docs.append(d)
        ids.append(h)
    return out_docs, ids, drops



def add_files_to_vectordb(filepath: str, reset: bool = False):
    if reset:
        reset_collection()

    # 1) Extraer
    raw_docs = extract_pdf_content(filepath)

    # 2) Chunking
    splits = chunk_documents(raw_docs)

    # 2.1) Dedupe por contenido
    unique_docs, ids, dropped = dedupe_by_hash(splits)

    # 3) Indexar con IDs deterministas
    vectorstore = Chroma.from_documents(
        documents=unique_docs,
        embedding=OpenAIEmbeddings(),
        persist_directory=VDB_PATH,
        collection_name=COLLECTION_NAME,
        ids=ids,  # 👈 clave
    )

    _safe_persist(vectorstore)

    # 4) Diagnóstico
    by_type = Counter(d.metadata.get("type", "text") for d in unique_docs)
    by_prio = Counter(d.metadata.get("prio", "general") for d in unique_docs)
    total_kv = sum(1 for d in unique_docs if "kv" in d.metadata)

    try:
        n_in_db = vectorstore._collection.count()
        print(f"📦 Documentos en DB (colección real): {n_in_db}")
    except Exception:
        n_in_db = None

    print(f"✅ {len(unique_docs)} fragmentos (tras dedupe; {dropped} descartados) indexados en {VDB_PATH} (colección '{COLLECTION_NAME}')")
    print("   📊 Por tipo:", dict(by_type))
    print("   📊 Por prioridad:", dict(by_prio))
    print(f"   🔑 Documentos con KV extraídos: {total_kv}")

    return vectorstore





In [7]:
# ============================================
# Paso 5 - Utilidad para inspeccionar colección
# ============================================
from collections import Counter
import chromadb

def get_collection_diagnostics(show_samples: int = 0):
    """
    Inspecciona la colección persistente y devuelve:
    - Fuentes únicas
    - Distribución por prioridad, tipo, páginas
    - Conteo de documentos con KV
    Opcional: muestra N ejemplos de metadatos
    """
    client = chromadb.PersistentClient(path=VDB_PATH)
    collection = client.get_collection(name=COLLECTION_NAME)

    data = collection.get(include=["documents", "metadatas"])
    metadatas = data.get("metadatas", [])
    documents = data.get("documents", [])

    sources = [m.get("source", "unknown").split("/")[-1] for m in metadatas if m]
    prio_counts = Counter(m.get("prio", "general") for m in metadatas if m)
    type_counts = Counter(m.get("type", "text") for m in metadatas if m)
    page_counts = Counter(m.get("page", "na") for m in metadatas if m)
    kv_total = sum(1 for m in metadatas if m and "kv" in m)

    print(f"📦 Colección: {COLLECTION_NAME}")
    print(f"📁 Fuentes únicas: {len(set(sources))} → {sorted(set(sources))}")
    print("   📊 Distribución por prioridad:", dict(prio_counts))
    print("   📊 Distribución por tipo:", dict(type_counts))
    print("   📊 Páginas únicas indexadas:", len(page_counts))
    print(f"   🔑 Documentos con KV: {kv_total}")

    if show_samples > 0:
        print("\n🔍 Ejemplos de metadatos:")
        for i, m in enumerate(metadatas[:show_samples], start=1):
            print(f"{i}.", m)

    return {
        "sources": sorted(set(sources)),
        "prio_counts": dict(prio_counts),
        "type_counts": dict(type_counts),
        "page_counts": dict(page_counts),
        "kv_total": kv_total,
        "total_docs": len(documents),
    }

# Ejemplo:
# resumen = get_collection_diagnostics(show_samples=3)
# print(resumen)


In [8]:
# ============================================
# Paso 5.1 - Chunking con preservación de metadatos
# ============================================
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document  # ✅ Import actualizado

def chunk_documents(
    docs: List[Document],
    chunk_size: int = 800,
    chunk_overlap: int = 100,
    verbose: bool = False
) -> List[Document]:
    """
    Divide documentos largos en chunks manejables para embeddings,
    conservando metadatos (source, page, type, prio, kv).
    
    - chunk_size: tamaño máximo de cada fragmento
    - chunk_overlap: solapamiento entre chunks
    - verbose: si True, imprime resumen por documento
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks: List[Document] = []

    for doc in docs:
        splits = splitter.split_text(doc.page_content)
        valid_splits = [s for s in splits if s.strip()]  # evitar basura vacía

        for idx, chunk in enumerate(valid_splits, start=1):
            new_meta = doc.metadata.copy()
            new_meta["chunk_id"] = f"{new_meta.get('page','na')}_{idx}"

            # Mantener prio y kv si existen
            if "prio" not in new_meta:
                new_meta["prio"] = "general"
            if "kv" in doc.metadata:
                new_meta["kv"] = doc.metadata["kv"]

            chunks.append(Document(page_content=chunk, metadata=new_meta))

        if verbose:
            print(f"📄 p.{doc.metadata.get('page','?')} "
                  f"| {doc.metadata.get('type','text')} "
                  f"→ {len(valid_splits)} chunks")

    if verbose:
        print(f"\n✅ Total de chunks generados: {len(chunks)}")

    return chunks



In [9]:
'''
# --- Paso 6: correr el pipeline ---
if __name__ == "__main__":
    # 🔄 Limpia todo si quieres partir desde cero:
    # reset_collection()

    vectorstore = add_files_to_vectordb(
        "./data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf",
        reset=False  # pon True si quieres borrar la colección antes
    )

    print("\n🚀 Pipeline completado. La colección está lista para consultas.")
'''

'\n# --- Paso 6: correr el pipeline ---\nif __name__ == "__main__":\n    # 🔄 Limpia todo si quieres partir desde cero:\n    # reset_collection()\n\n    vectorstore = add_files_to_vectordb(\n        "./data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf",\n        reset=False  # pon True si quieres borrar la colección antes\n    )\n\n    print("\n🚀 Pipeline completado. La colección está lista para consultas.")\n'

In [10]:
# --- Paso 6: correr el pipeline ---
if __name__ == "__main__":
    #reset_collection(full_reset=True)   # reinicia todo

    vectorstore = add_files_to_vectordb(
        "./data/FRDFMEQ450. 1 (Requisicion de Materiales CIP POZO HUFF&PUFF).pdf",
        reset=True  # pon True si quieres borrar la colección antes
    )

    print("\n🚀 Pipeline completado. La colección está lista para consultas.")


🗑️ Colección eliminada en ./vectordb
📦 Documentos en DB (colección real): 209
✅ 209 fragmentos (tras dedupe; 0 descartados) indexados en ./vectordb (colección 'autoselx_docs')
   📊 Por tipo: {'text': 92, 'table': 117}
   📊 Por prioridad: {'alcance': 21, 'norma': 63, 'general': 52, 'hoja_datos': 32, 'requisito': 41}
   🔑 Documentos con KV extraídos: 17

🚀 Pipeline completado. La colección está lista para consultas.


In [11]:
if __name__ == "__main__":
    # 🔄 Limpia todo si quieres partir desde cero:
    # reset_collection()

    vectorstore = add_files_to_vectordb(
        "./data/FRDFMEX450. 1 (Hoja de Datos CIP POZO HUFF&PUFF).pdf",
        reset=False  # pon True si quieres borrar la colección antes
    )

    print("\n🚀 Pipeline completado. La colección está lista para consultas.")

📦 Documentos en DB (colección real): 248
✅ 39 fragmentos (tras dedupe; 0 descartados) indexados en ./vectordb (colección 'autoselx_docs')
   📊 Por tipo: {'text': 13, 'table': 26}
   📊 Por prioridad: {'hoja_datos': 38, 'general': 1}
   🔑 Documentos con KV extraídos: 33

🚀 Pipeline completado. La colección está lista para consultas.


In [12]:
if __name__ == "__main__":
    # 🔄 Limpia todo si quieres partir desde cero:
    # reset_collection()

    vectorstore = add_files_to_vectordb(
        "./data/FRDFICW450.1listado de señales.pdf",
        reset=False  # pon True si quieres borrar la colección antes
    )

    print("\n🚀 Pipeline completado. La colección está lista para consultas.")

📦 Documentos en DB (colección real): 332
✅ 84 fragmentos (tras dedupe; 0 descartados) indexados en ./vectordb (colección 'autoselx_docs')
   📊 Por tipo: {'table': 50, 'text': 34}
   📊 Por prioridad: {'general': 84}
   🔑 Documentos con KV extraídos: 61

🚀 Pipeline completado. La colección está lista para consultas.


In [13]:
if __name__ == "__main__":
    # 🔄 Limpia todo si quieres partir desde cero:
    # reset_collection()

    vectorstore = add_files_to_vectordb(
        "./data/FRDFMEW450. 1 (Especificacion Tecnica CIP POZO HUFF&PUFF).pdf",
        reset=False  # pon True si quieres borrar la colección antes
    )

    print("\n🚀 Pipeline completado. La colección está lista para consultas.")

📦 Documentos en DB (colección real): 521
✅ 189 fragmentos (tras dedupe; 2 descartados) indexados en ./vectordb (colección 'autoselx_docs')
   📊 Por tipo: {'text': 127, 'table': 62}
   📊 Por prioridad: {'general': 61, 'requisito': 72, 'alcance': 16, 'norma': 11, 'hoja_datos': 29}
   🔑 Documentos con KV extraídos: 20

🚀 Pipeline completado. La colección está lista para consultas.


In [14]:
# ============================================
# Verificar qué documentos hay en vectordb
# ============================================
import chromadb

# Conectar al cliente persistente en la ruta que estás usando
PERSIST_DIR = "/home/user/Desktop/Tesis2025/AutoSelectX/scriptsSampleRAG/vectordb"
client = chromadb.PersistentClient(path=PERSIST_DIR)

# Nombre de la colección que usaste al crear los embeddings
COLLECTION_NAME = "autoselx_docs"

def get_unique_sources_list(client, collection_name):
    try:
        collection = client.get_collection(collection_name)
        # Obtener todos los documentos y metadatos
        data = collection.get(include=["metadatas"])
        metadatas = data.get("metadatas", [])
        
        sources = set()
        for metadata in metadatas:
            if metadata and "source" in metadata:
                sources.add(metadata["source"])
        
        # Extraer solo el nombre de archivo
        file_names = sorted(set(source.split("/")[-1] for source in sources))
        return file_names
    except Exception as e:
        return f"⚠️ Error al acceder a la colección: {e}"

# Ejecutar
docs_in_vdb = get_unique_sources_list(client, COLLECTION_NAME)
print("📂 Documentos en vectordb:", docs_in_vdb)



📂 Documentos en vectordb: ['FRDFICW450.1listado de señales.pdf', 'FRDFMEQ450. 1 (Requisicion de Materiales CIP POZO HUFF&PUFF).pdf', 'FRDFMEW450. 1 (Especificacion Tecnica CIP POZO HUFF&PUFF).pdf', 'FRDFMEX450. 1 (Hoja de Datos CIP POZO HUFF&PUFF).pdf']


In [15]:
# ============================================
# Paso 7 - Debug robusto: explorar fragmentos (MMR con fallback, mejora alcance) TEST!!!!!!!!!!!!!!!!!!!!!!!
# ============================================
import json

def _try_retrieve(query: str, search_kwargs: dict, use_mmr: bool = True):
    """Envuelve as_retriever con fallback y logs amigables."""
    try:
        retriever = vectorstore.as_retriever(
            search_type="mmr" if use_mmr else "similarity",
            search_kwargs=search_kwargs,
        )
        return retriever.get_relevant_documents(query)
    except Exception as e:
        modo = "MMR" if use_mmr else "similarity"
        print(f"(i) Falló {modo} con error {e.__class__.__name__}: {e}")
        return []

def debug_busqueda_alcance(
    q: str = "¿Cuál es el alcance del suministro?",
    k_text: int = 5,
    k_mixed: int = 5,
    preferir_texto: bool = True,
):
    print(f"🔎 Consulta: {q}")

    # --- 0) Chequeo rápido de colección
    try:
        coll = vectorstore._collection
        _probe = coll.get(limit=3)
        if not _probe.get("ids"):
            print("(i) Colección vacía o corrupta; considera resetear y reindexar.")
    except Exception as e:
        print(f"(i) No pude inspeccionar la colección: {e}")

    # --- 1) Fase A: priorizar TEXTO con prio ∈ {alcance, general, hoja_datos}
    kwargs_a = {
        "k": max(k_text, 5),
        "fetch_k": 40,
        "lambda_mult": 0.4,
        "filter": {"prio": {"$in": ["alcance", "general", "hoja_datos"]}},  # 👈 ampliado
    }

    # Intento MMR
    res_text = _try_retrieve(q, kwargs_a, use_mmr=True)
    # Fallback similarity
    if not res_text:
        kwargs_a_fallback = {
            "k": max(k_text, 5),
            "filter": {"prio": {"$in": ["alcance", "general", "hoja_datos"]}},
        }
        res_text = _try_retrieve(q, kwargs_a_fallback, use_mmr=False)

    # Post-filtro por tipo en Python
    if preferir_texto:
        res_text = [r for r in res_text if r.metadata.get("type") == "text"]

    print(f"\n📚 TEXT preferido (alcance/general/hoja_datos) → {len(res_text)}")

    # --- Mostrar únicos
    seen_hash = set()
    def _mark_and_show(prefix: str, doc, idx: int):
        ch = doc.metadata.get("content_hash") or f"{doc.metadata.get('page')}_{doc.metadata.get('chunk_id')}"
        if ch in seen_hash:
            return False
        seen_hash.add(ch)
        print(f"\n--- {prefix} {idx} ---")
        print((doc.page_content[:400]).replace("\n", " "))
        print("Metadatos:", {k: doc.metadata.get(k) for k in ["page","type","prio","source","chunk_id"]})
        if "kv" in doc.metadata:
            try:
                print("KV:", json.loads(doc.metadata["kv"]))
            except Exception:
                print("KV (raw):", doc.metadata["kv"])
        return True

    shown = 0
    for i, r in enumerate(res_text, 1):
        if _mark_and_show("TEXT", r, i):
            shown += 1

    # --- 2) Fase B: mezcla (text+table) como respaldo
    kwargs_b = {"k": max(k_mixed, 5)}
    res_mixed = _try_retrieve(q, kwargs_b, use_mmr=False)  # similarity estable
    print(f"\n🧩 MIX (text+table) → {len(res_mixed)}")

    added = 0
    for r in res_mixed:
        if _mark_and_show("MIX", r, added + 1):
            added += 1
        if added >= k_mixed:
            break

    print(f"\n📌 Total únicos mostrados: {len(seen_hash)}")


# 🚀 Ejecutar debug
debug_busqueda_alcance()


🔎 Consulta: ¿Cuál es el alcance del suministro?


/tmp/ipykernel_6614/2087002342.py:13: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return retriever.get_relevant_documents(query)



📚 TEXT preferido (alcance/general/hoja_datos) → 3

--- TEXT 1 ---
“Formato de solicitud técnicas”. 2. ALCANCE DEL SUMINISTRO 2.1. ALCANCE GENERAL A continuación, se listan el paquete objeto de esta requisición de materiales. El PROVEEDOR debe incluir los equipos, personal, herramientas, dispositivos, elementos y materiales necesarios y suficientes para que el PAQUETE sea funcional y opere adecuadamente bajo las condiciones indicadas en los documentos anexos que 
Metadatos: {'page': 5, 'type': 'text', 'prio': 'alcance', 'source': './data/FRDFMEQ450. 1 (Requisicion de Materiales CIP POZO HUFF&PUFF).pdf', 'chunk_id': '5_4'}

--- TEXT 2 ---
alcance del suministro del PAQUETE podrán estar en idioma inglés y/o español, a excepción de los manuales de preservación, instalación, operación y mantenimiento que deberán ser elaborados en idioma español.
Metadatos: {'page': 17, 'type': 'text', 'prio': 'hoja_datos', 'source': './data/FRDFMEW450. 1 (Especificacion Tecnica CIP POZO HUFF&PUFF).pdf', 'c

In [16]:
# ==========================
# Complemento Paso 7 - Función auxiliar de debug
# ==========================
def debug_query(vectorstore, query, prio=None, k=5):
    """
    Ejecuta una búsqueda con filtro de metadatos (cuando prio != None).
    Incluye fallbacks para prioridades relacionadas y muestra resultados
    con metadatos enriquecidos y KV cuando existen.
    """
    import json

    # Prios relacionadas para fallback (más tolerancia)
    prio_fallbacks = {
        "alcance":    ["alcance", "general", "hoja_datos"],
        "requisito":  ["requisito", "hoja_datos", "general"],
        "norma":      ["norma", "general"],
        "hoja_datos": ["hoja_datos", "requisito", "general"],
        "general":    ["general", "alcance", "hoja_datos"]
    }

    print(f"\n🔎 DEBUG QUERY: '{query}' (prio={prio})\n")

    def run_search(allowed_prios, topk):
        if allowed_prios is None:
            # búsqueda sin filtro
            return vectorstore.similarity_search(query, k=topk)
        else:
            # búsqueda con filtro por metadatos
            return vectorstore.similarity_search(
                query,
                k=topk,
                filter={"prio": {"$in": allowed_prios}}
            )

    # --- 1) Intento principal con filtro si prio está definido
    if prio:
        allowed = prio_fallbacks.get(prio, [prio])
        docs = run_search(allowed_prios=allowed, topk=max(k, 10))

        # --- 2) Fallback si no hay resultados
        if not docs:
            allowed_wide = list(set(allowed + ["general"]))
            docs = run_search(allowed_prios=allowed_wide, topk=50)

        if not docs:
            print("⚠️ No se encontraron resultados con esa prioridad (ni con fallback).")
            return
    else:
        # búsqueda sin restricción de prioridad
        docs = run_search(allowed_prios=None, topk=k)

    # --- Mostrar resultados
    print(f"📊 Fragmentos recuperados: {len(docs)}\n")
    seen = set()
    for i, r in enumerate(docs, 1):
        ch = r.metadata.get("content_hash") or f"{r.metadata.get('page')}_{r.metadata.get('chunk_id')}"
        if ch in seen:
            continue
        seen.add(ch)

        print(f"--- CHUNK {i} ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", {k: r.metadata.get(k) for k in ["page","type","prio","source","chunk_id"]})
        if "kv" in r.metadata:
            try:
                print("KV dict:", json.loads(r.metadata["kv"]))
            except Exception:
                print("KV (raw):", r.metadata["kv"])
        print()

    print(f"📌 Total únicos mostrados: {len(seen)}")


In [17]:
# ==========================
# Pruebas de debug por prioridad
# ==========================

# Alcance del suministro
debug_query(vectorstore, "alcance del suministro", prio="alcance")


🔎 DEBUG QUERY: 'alcance del suministro' (prio=alcance)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
alcance del suministro del PAQUETE podrán estar en idioma inglés y/o español, a excepción de los manuales de preservación, instalación, operación y mantenimiento que deberán ser elaborados en idioma español.
Metadatos: {'page': 17, 'type': 'text', 'prio': 'hoja_datos', 'source': './data/FRDFMEW450. 1 (Especificacion Tecnica CIP POZO HUFF&PUFF).pdf', 'chunk_id': '17_4'}

--- CHUNK 2 ---
a suministrar, como también los detalles relacionados con los requisitos del proceso de construcción, características de desempeño y algunos otros requerimientos, como también todos los sistemas auxiliares asociados con el sistema que deberán formar parte integral del alcance del PROVEEDOR de este suministro descritos en la sección 7 de esta especificación y en la Requisición de Materiales No. FRD
Metadatos: {'page': 8, 'type': 'text', 'prio': 'alcance', 'source': './data/FRDFMEW450. 1 (Especificacion T

In [18]:
debug_query(vectorstore, "API 675 caudal presión material", prio="hoja_datos")


🔎 DEBUG QUERY: 'API 675 caudal presión material' (prio=hoja_datos)



📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
INDICADOENESTEDOCUMENTO;SINEMBARGO,ELPROVEEDORDELPAQUETEDEBERÁCONFIRMARESTOSDATOS(INCLUYENDOLACARACTERIZACIÓNDELPUNTODEINYECCIÓN)DEFORMACONJUNTACONELPROVEEDORDEQUÍMICOSESTABLECIDOPOR ECOPETROLPARAELSUMINISTRODELMISMO.ESTOCONELFINDEASEGURARUNADECUADOPROCESODEPROTECCIÓNDELASSUPERFICIESMETÁLICASCONTRAELATAQUEQUÍMICOPORAGUAYOTROSCONTAMINANTESCONELOBJETIVODEDISMINUIR LAVELOCIDADDECORROSIÓN.ADICIONALMEN
Metadatos: {'page': 3, 'type': 'table', 'prio': 'hoja_datos', 'source': './data/FRDFMEX450. 1 (Hoja de Datos CIP POZO HUFF&PUFF).pdf', 'chunk_id': '3_7'}
KV dict: {'presion_psig': '50', 'material_tanque': '304SS', 'api_675': 'true'}

--- CHUNK 2 ---
LAVELOCIDADDECORROSIÓN.ADICIONALMENTE,CONELFINDEGARANTIZARFLEXIBILIDADENELCAUDALASERDOSIFICADO,LASBOMBASDEBERÁNTENERUNTURNDOWNRATIODE10:1ENELCUALELCAUDALDEBERÁTENERUNAPRECISIÓNDEL±1% SEGÚN LO INDICADO EN LA API 675. 47 13. LA PRESIÓN DE DESCARGA GARANTIZA 50 PSIG POR ENCIMA DE LA PRESION EN LA LINEA DE

In [19]:
debug_query(vectorstore, "RETIE y normas aplicables", prio="norma")


🔎 DEBUG QUERY: 'RETIE y normas aplicables' (prio=norma)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
Certificado de conformidad de producto Según RETIE para el motor eléctrico. INFO E 3+E cronograma (Para motores mayores a 375 W) Protocolo certificado de resultado de Según INFO E E pruebas de rutina del motor. cronograma Términos de entrega. x - IFR E E Garantías x +2D IFR E E NOTAS (1) (*) DOCUMENTOS PRIORITARIOS SUJETOS A PENALIZACIÓN O TÉRMINOS DE PAGO (2) FECHA DE EMISIÓN: “+ “significa Días 
Metadatos: {'page': 10, 'type': 'text', 'prio': 'norma', 'source': './data/FRDFMEQ450. 1 (Requisicion de Materiales CIP POZO HUFF&PUFF).pdf', 'chunk_id': '10_3'}

--- CHUNK 2 ---
internas, etc).  | | | | | | R | R | | | | | | | |   | | | | | | R | R | | | | | | | |   | | | | | | R | R | | | | | | | |   | | | | | | R | R | | | | | | | |  NÓICACIRBAF ETNARUD | | | | | | | | | | | | | | |  ABEURP LANIF | Revisión de resultados de las pruebas. Revisión de características de los componentes Revi

In [20]:
debug_query(vectorstore, "requisitos técnicos del sistema de dosificación", prio="requisito")


🔎 DEBUG QUERY: 'requisitos técnicos del sistema de dosificación' (prio=requisito)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
químicos de este, es decir diseño, fabricación y suministro de forma modular (uno para la bomba y otro para tanque). El PROVEEDOR deberá enviar planos de detalle de las boquillas conformadas con el cuerpo del tanque. El tanque de almacenamiento de químicos deberá tener su propia placa de identificación independiente, indicando: fabricante, capacidad, tamaño, modelo, fecha de fabricación. 7.4.2.2. 
Metadatos: {'page': 18, 'type': 'text', 'prio': 'requisito', 'source': './data/FRDFMEW450. 1 (Especificacion Tecnica CIP POZO HUFF&PUFF).pdf', 'chunk_id': '18_3'}
KV dict: {'material_tanque': '304SS'}

--- CHUNK 2 ---
sección 6.2 tabla 3. El variador deberá ser montado y probado en fabrica con la bomba-motor dosificadora a las condiciones de operación indicadas en la hoja de datos No. FRD/F-MEX-450. 7.4.2.4. Placa de identificación La placa de identificación del pa

In [21]:
# ============================================
# Paso 8 - Probar extracción de Hoja de Datos API 675
# ============================================
import json

def buscar_hojas_datos(pregunta: str = "hoja de datos bomba", k: int = 12):
    # --- 1) Buscamos primero chunks con prio=hoja_datos ---
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    results = retriever.get_relevant_documents(pregunta)
    hd_chunks = [r for r in results if r.metadata.get("prio") == "hoja_datos"]

    # --- 2) Si hay pocos resultados, ampliamos con los que tengan KV ---
    if len(hd_chunks) < 3:
        kv_chunks = [r for r in results if "kv" in r.metadata]
        filtrados = hd_chunks + kv_chunks
    else:
        filtrados = hd_chunks

    print(f"🔎 Total de chunks candidatos (HD/kv): {len(filtrados)}")

    count = 0
    for r in filtrados:
        has_kv = "kv" in r.metadata
        if has_kv:
            count += 1

        flag = "🔑" if has_kv else ""
        print(f"\n--- CHUNK HD {flag} ---")
        print(f"p.{r.metadata.get('page')} | {r.metadata.get('type')} | prio={r.metadata.get('prio')}")
        print(r.page_content[:400])

        if has_kv:
            try:
                kv = json.loads(r.metadata["kv"])
                print("KV:", kv)
            except Exception:
                print("KV (raw):", r.metadata["kv"])

    print(f"✅ Chunks con KV reales: {count}")
    return filtrados


# Ejemplo:
buscar_hojas_datos("Datos de diseño de bomba dosificadora API 675 (caudal, presión, HP)")



🔎 Total de chunks candidatos (HD/kv): 11

--- CHUNK HD 🔑 ---
p.2 | table | prio=hoja_datos
m Quill de inyección con boquilla de dosificación
l El vendedor suministra válvula de alivio de presión (NOTA 11)
m Interna a la bomba mExterna a la bomba
o Presión disparo válvula de alivio (psig) POR PROVEEDOR
l Indicador de presión a la descarga de las bombas (NOTA 11)
l Válvulas dobles de antirretorno requeridas
l Tablero eléctrico (mínimo NEMA 4X + 7) 1
l Sistema de Control (NOTA 8)
l Indicad
KV: {'hp_motor': '3', 'material_tanque': '304SS', 'material_mojado': '316SS', 'api_675': 'true'}

--- CHUNK HD 🔑 ---
p.2 | table | prio=hoja_datos
Número | | | | | | | | 
46 | | | | | | | | | 
47 | | | | | ESPECIFICACIONES APLICABLES: | | | | 
48 | | | | | l API 675 - POSITIVE DISP. PUMPS - CONTROLLED VOLUME
m ATEX (94/9/EC) Grupo Categ. Clase temp.
m Otra: | | | | 
49 | | | | | | | | | 
50 | | | | | | | | | 
1 | o MATERIALES BOMBAS (NOTA 1) | | | | CONTROL (NOTA 8) | | | | 
2 | Cabezal hidráulico 316 S

[Document(id='f4f07da5f85092402cb66e4b5e456ede09e106ac', metadata={'source': './data/FRDFMEX450. 1 (Hoja de Datos CIP POZO HUFF&PUFF).pdf', 'page': 2, 'prio': 'hoja_datos', 'type': 'table', 'chunk_id': '2_9', 'kv': '{"hp_motor": "3", "material_tanque": "304SS", "material_mojado": "316SS", "api_675": "true"}', 'content_hash': 'f4f07da5f85092402cb66e4b5e456ede09e106ac'}, page_content='m Quill de inyección con boquilla de dosificación\nl El vendedor suministra válvula de alivio de presión (NOTA 11)\nm Interna a la bomba mExterna a la bomba\no Presión disparo válvula de alivio (psig) POR PROVEEDOR\nl Indicador de presión a la descarga de las bombas (NOTA 11)\nl Válvulas dobles de antirretorno requeridas\nl Tablero eléctrico (mínimo NEMA 4X + 7) 1\nl Sistema de Control (NOTA 8)\nl Indicador de Nivel, Transmisor de Nivel en el Tanque\nl Skid estructural para paquetizado MATERIAL: SS304 Anclaje y Montaje: En fundaciones con placa base, pernos de anclaje\nl Válvula de alivio de presión diferen

In [22]:
# ============================================
# Paso 8.1 - Consolidar Hoja de Datos API 675
# ============================================
import json
from typing import Dict, Any


def extraer_campos_extras(texto: str) -> Dict[str, str]:
    """Regex para campos no capturados aún en kv_from_text_page"""
    t = texto.upper()
    kv = {}

    # Fluido de operación
    m = re.search(r"FLUIDO\s+DE\s+OPERACI[ÓO]N\s+SI\s+([A-ZÁÉÍÓÚ0-9\s]+)", t)
    if m:
        kv["fluido"] = m.group(1).title()

    # Viscosidad cP
    m = re.search(r"VISCO(SIDAD)?[^0-9]*(\d{2,5})\s*-\s*(\d{2,5})\s*CP", t)
    if m:
        kv["viscosidad_cps"] = f"{m.group(2)}-{m.group(3)}"

    # Eficiencia %
    m = re.search(r"EFICIENCIA.*?(\d{1,3})\s*%|≥\s*(\d{1,3})", t)
    if m:
        kv["eficiencia_pct"] = m.group(1) or m.group(2)

    # Clasificación de área
    m = re.search(r"CLASIFICACI[ÓO]N\s+DEL\s+ÁREA\s+NO\s+([A-Z\s]+)", t)
    if m:
        kv["clasificacion_area"] = m.group(1).title()

    return kv

def consolidar_hoja_datos(pregunta="hoja de datos bomba API 675", k=12) -> Dict[str, Any]:
    docs = buscar_hojas_datos(pregunta, k=k)
    consolidado = {}

    for d in docs:
        # 1) KV en metadata
        if "kv" in d.metadata:
            try:
                kv = json.loads(d.metadata["kv"])
                consolidado.update(kv)
            except Exception:
                pass

        # 2) Extra campos con regex sobre el contenido
        extras = extraer_campos_extras(d.page_content)
        consolidado.update(extras)

    print("\n✅ JSON consolidado:")
    print(json.dumps(consolidado, indent=2, ensure_ascii=False))
    return consolidado

# Ejemplo:
consolidar_hoja_datos("Datos de diseño de bomba dosificadora API 675")


🔎 Total de chunks candidatos (HD/kv): 11

--- CHUNK HD 🔑 ---
p.2 | table | prio=hoja_datos
Número | | | | | | | | 
46 | | | | | | | | | 
47 | | | | | ESPECIFICACIONES APLICABLES: | | | | 
48 | | | | | l API 675 - POSITIVE DISP. PUMPS - CONTROLLED VOLUME
m ATEX (94/9/EC) Grupo Categ. Clase temp.
m Otra: | | | | 
49 | | | | | | | | | 
50 | | | | | | | | | 
1 | o MATERIALES BOMBAS (NOTA 1) | | | | CONTROL (NOTA 8) | | | | 
2 | Cabezal hidráulico 316 SS
Placas de contorno
Diafragma
Émbolo N
KV: {'hp_motor': '3', 'material_tanque': '304SS', 'material_mojado': '316SS', 'api_675': 'true'}

--- CHUNK HD 🔑 ---
p.3 | table | prio=hoja_datos
SITIO DE INSTALACIÓN Y LOS ESTÁNDARES DE CALIDAD DE ECOPETROL, GARANTIZANDO LA INTEGRIDAD Y DESEMPEÑO DE LAS BOMBAS DURANTE SU OPERACIÓN. | | | 
36 | 2.ELPROVEEDORDEBECOMPLETARESTAHOJADEDATOS,VERIFICARLOSDATOSSUMINISTRADOSEINDICARLAMEJORALTERNATIVADECADAÍTEM.ENCASODEQUEEXISTANDESVIACIONESCONRESPECTOALAINFORMACIÓNESPECIFICADAENESTE
DOCUMENTOELPROVEEDORDEBERÁPR

{'hp_motor': '3',
 'material_tanque': '304SS',
 'material_mojado': '316SS',
 'api_675': 'true',
 'presion_psig': '50'}

In [23]:
# ============================================
# Paso 9 - Inspeccionar metadatos del Alcance
# ============================================

def buscar_alcance(pregunta: str = "alcance del suministro", k: int = 8):
    """Busca chunks relacionados con alcance (prio=alcance) y muestra metadatos"""
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k, "filter": {"prio": "alcance"}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Total de chunks recuperados (prio=alcance): {len(results)}")
    for r in results:
        print(
            f"\n--- CHUNK p.{r.metadata.get('page')} | "
            f"{r.metadata.get('type')} | prio={r.metadata.get('prio')} ---"
        )
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

    return results

# Ejemplo de prueba:
buscar_alcance()



🔎 Total de chunks recuperados (prio=alcance): 8

--- CHUNK p.8 | text | prio=alcance ---
a suministrar, como también los detalles relacionados con los requisitos del proceso de construcción, características de desempeño y algunos otros requerimientos, como también todos los sistemas auxiliares asociados con el sistema que deberán formar parte integral del alcance del PROVEEDOR de este suministro descritos en la sección 7 de esta especificación y en la Requisición de Materiales No. FRD
Metadatos: {'chunk_id': '8_2', 'type': 'text', 'page': 8, 'prio': 'alcance', 'content_hash': 'dd2e96433f619dc7ee0447b50b1f3a73c5941428', 'source': './data/FRDFMEW450. 1 (Especificacion Tecnica CIP POZO HUFF&PUFF).pdf'}

--- CHUNK p.5 | text | prio=alcance ---
“Formato de solicitud técnicas”. 2. ALCANCE DEL SUMINISTRO 2.1. ALCANCE GENERAL A continuación, se listan el paquete objeto de esta requisición de materiales. El PROVEEDOR debe incluir los equipos, personal, herramientas, dispositivos, elementos y ma

[Document(id='dd2e96433f619dc7ee0447b50b1f3a73c5941428', metadata={'chunk_id': '8_2', 'type': 'text', 'page': 8, 'prio': 'alcance', 'content_hash': 'dd2e96433f619dc7ee0447b50b1f3a73c5941428', 'source': './data/FRDFMEW450. 1 (Especificacion Tecnica CIP POZO HUFF&PUFF).pdf'}, page_content='a suministrar, como también los detalles relacionados con los requisitos del proceso de construcción,\ncaracterísticas de desempeño y algunos otros requerimientos, como también todos los sistemas auxiliares\nasociados con el sistema que deberán formar parte integral del alcance del PROVEEDOR de este suministro\ndescritos en la sección 7 de esta especificación y en la Requisición de Materiales No. FRD/F-MEQ-450.\nEl PROVEEDOR deberá suministrar, probar y entregar en funcionamiento, el sistema de inyección de química\ntotalmente nuevo y con todos los accesorios, insumos y equipos asociados para su correcto funcionamiento.'),
 Document(id='03d07d97e335b04d4564e35cd0f4564ff7ad85a7', metadata={'chunk_id': '

In [24]:
# ============================================
# Paso 10 - Validar TODOS los metadatos (corregido)
# ============================================
from collections import Counter

def validar_metadatos_globales():
    """Valida consistencia de metadatos en la colección completa."""
    collection = vectorstore._collection
    data = collection.get(include=["metadatas", "documents"])

    total_docs = len(data["documents"])
    print(f"🔎 Total de registros en la colección: {total_docs}")

    errores = 0
    prio_counts = Counter()
    type_counts = Counter()
    kv_count = 0
    seen_ids = set()
    duplicados = 0

    for i, meta in enumerate(data["metadatas"]):
        doc_id = data["ids"][i]  # aquí sí podemos usarlo, ya viene siempre

        # Detectar duplicados de ID
        if doc_id in seen_ids:
            duplicados += 1
        else:
            seen_ids.add(doc_id)

        # Validar claves mínimas
        if not all(key in meta for key in ["source", "page", "type"]):
            print(f"⚠️ Registro {i} con metadatos incompletos:", meta)
            errores += 1
            continue

        # Validar valores
        if not isinstance(meta.get("page"), int) or meta["page"] <= 0:
            print(f"⚠️ Registro {i} con página inválida:", meta)
            errores += 1

        if not meta.get("type"):
            print(f"⚠️ Registro {i} con tipo vacío:", meta)
            errores += 1

        # Contadores útiles
        prio_counts[meta.get("prio", "none")] += 1
        type_counts[meta.get("type", "unknown")] += 1
        if "kv" in meta:
            kv_count += 1

    # Resumen
    if errores == 0:
        print("✅ Todos los documentos tienen metadatos completos y válidos.")
    else:
        print(f"⚠️ Se encontraron {errores} problemas de metadatos.")

    if duplicados > 0:
        print(f"⚠️ Se detectaron {duplicados} duplicados de IDs.")

    print("\n📊 Distribución por prioridad:", dict(prio_counts))
    print("📊 Distribución por tipo:", dict(type_counts))
    print(f"📊 Total de documentos con KV (tablas clave-valor): {kv_count}")

# Ejemplo:
validar_metadatos_globales()



🔎 Total de registros en la colección: 521
✅ Todos los documentos tienen metadatos completos y válidos.

📊 Distribución por prioridad: {'alcance': 37, 'norma': 74, 'general': 198, 'hoja_datos': 99, 'requisito': 113}
📊 Distribución por tipo: {'text': 266, 'table': 255}
📊 Total de documentos con KV (tablas clave-valor): 131


In [25]:
# ============================================
# Paso 11 - Buscar solo tablas o solo texto
# ============================================
def buscar_por_tipo(tipo: str = "table", pregunta: str = "bomba dosificadora", k: int = 5):
    """
    Recupera fragmentos filtrando por tipo ('table' o 'text').
    """
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k, "filter": {"type": tipo}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"\n🔎 Resultados filtrados por tipo='{tipo}' | pregunta='{pregunta}' → {len(results)} encontrados")
    for i, r in enumerate(results, 1):
        print(f"\n--- CHUNK {i} ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

        if "kv" in r.metadata:  # Mostrar KV si existe
            try:
                import json
                print("KV:", json.loads(r.metadata["kv"]))
            except Exception:
                print("KV (raw):", r.metadata["kv"])

    return results


# Ejemplos de prueba:
buscar_por_tipo("table", "tags y cantidades")
buscar_por_tipo("text", "alcance del suministro")




🔎 Resultados filtrados por tipo='table' | pregunta='tags y cantidades' → 5 encontrados

--- CHUNK 1 ---
Ítem | Cantidad | Unidad | TAG | Descripción | Código de Catálogo ECOPETROL 1 | 1 | un | I0-CIP-42001 | Paquete de Inyección de Químicos (Inhibidor de Corrosión) | N/A
Metadatos: {'source': './data/FRDFMEQ450. 1 (Requisicion de Materiales CIP POZO HUFF&PUFF).pdf', 'type': 'table', 'page': 5, 'chunk_id': '5_1', 'content_hash': 'ce4d1c93f114618f4aed9d12332522ff66e74910', 'prio': 'norma'}

--- CHUNK 2 ---
Ítem | Cantidad | Unidad | TAG | Descripción | Código de Catálogo ECOPETROL 1 | 1 | UN | I0/CIP/42 001 | suministro de Unidad Paquete de Inyección de Químicos, Incluye (sin limitarse a): - Un (1) conjunto motor eléctrico y bomba dosificadora tipo monobloque de volumen controlado API 675 para inyección de químico. - Un (1) tanque Isotanque de fabricación en acero inoxidable 304 SS con capacidad de 
Metadatos: {'page': 32, 'chunk_id': '32_1', 'source': './data/FRDFMEQ450. 1 (Requisicion

[Document(id='f4aa8444b6ab3d43fad425b32e3daecada9dcc16', metadata={'chunk_id': '17_4', 'prio': 'hoja_datos', 'type': 'text', 'page': 17, 'content_hash': 'f4aa8444b6ab3d43fad425b32e3daecada9dcc16', 'source': './data/FRDFMEW450. 1 (Especificacion Tecnica CIP POZO HUFF&PUFF).pdf'}, page_content='alcance del suministro del PAQUETE podrán estar en idioma inglés y/o español, a excepción de los manuales\nde preservación, instalación, operación y mantenimiento que deberán ser elaborados en idioma español.'),
 Document(id='dd2e96433f619dc7ee0447b50b1f3a73c5941428', metadata={'content_hash': 'dd2e96433f619dc7ee0447b50b1f3a73c5941428', 'chunk_id': '8_2', 'source': './data/FRDFMEW450. 1 (Especificacion Tecnica CIP POZO HUFF&PUFF).pdf', 'page': 8, 'type': 'text', 'prio': 'alcance'}, page_content='a suministrar, como también los detalles relacionados con los requisitos del proceso de construcción,\ncaracterísticas de desempeño y algunos otros requerimientos, como también todos los sistemas auxiliare

In [26]:
# ============================================
# Paso 11.1 - Comparar resultados table vs text
# ============================================
from collections import Counter
import json

def comparar_table_vs_text(pregunta="alcance del suministro", k=6):
    resultados = {}
    for tipo in ["table", "text"]:
        retriever = vectorstore.as_retriever(
            search_kwargs={"k": k, "filter": {"type": tipo}}
        )
        docs = retriever.get_relevant_documents(pregunta)
        resultados[tipo] = docs

        print(f"\n🔎 Resultados filtrados por tipo='{tipo}' → {len(docs)} encontrados")
        prio_counter = Counter([d.metadata.get("prio", "none") for d in docs])
        kv_count = sum(1 for d in docs if "kv" in d.metadata)

        print(f"📊 Distribución de prioridades: {dict(prio_counter)}")
        print(f"📊 Con KV extraídos: {kv_count}")

        for r in docs:
            print("\n--- CHUNK ---")
            print(r.page_content[:300].replace("\n", " "))
            print("Metadatos:", r.metadata)
            if "kv" in r.metadata:
                try:
                    print("KV:", json.loads(r.metadata["kv"]))
                except:
                    print("KV (raw):", r.metadata["kv"])

    # Comparación final
    print("\n================= RESUMEN COMPARATIVO =================")
    for tipo in ["table", "text"]:
        print(f"Tipo: {tipo} | Total: {len(resultados[tipo])}")
        kv_total = sum(1 for d in resultados[tipo] if "kv" in d.metadata)
        print(f" → Con KV: {kv_total}")
        prio_counter = Counter([d.metadata.get("prio", "none") for d in resultados[tipo]])
        print(f" → Prioridades: {dict(prio_counter)}")

# Ejemplo de prueba
comparar_table_vs_text("tags y cantidades", k=5)
comparar_table_vs_text("alcance del suministro", k=5)



🔎 Resultados filtrados por tipo='table' → 5 encontrados
📊 Distribución de prioridades: {'norma': 2, 'hoja_datos': 2, 'general': 1}
📊 Con KV extraídos: 2

--- CHUNK ---
Ítem | Cantidad | Unidad | TAG | Descripción | Código de Catálogo ECOPETROL 1 | 1 | un | I0-CIP-42001 | Paquete de Inyección de Químicos (Inhibidor de Corrosión) | N/A
Metadatos: {'page': 5, 'prio': 'norma', 'chunk_id': '5_1', 'source': './data/FRDFMEQ450. 1 (Requisicion de Materiales CIP POZO HUFF&PUFF).pdf', 'type': 'table', 'content_hash': 'ce4d1c93f114618f4aed9d12332522ff66e74910'}

--- CHUNK ---
Ítem | Cantidad | Unidad | TAG | Descripción | Código de Catálogo ECOPETROL 1 | 1 | UN | I0/CIP/42 001 | suministro de Unidad Paquete de Inyección de Químicos, Incluye (sin limitarse a): - Un (1) conjunto motor eléctrico y bomba dosificadora tipo monobloque de volumen controlado API 675 para inyecci
Metadatos: {'chunk_id': '32_1', 'source': './data/FRDFMEQ450. 1 (Requisicion de Materiales CIP POZO HUFF&PUFF).pdf', 'page': 3

In [27]:
# ============================================
# Paso 12 - Buscar en una página específica
# ============================================
def buscar_por_pagina(pagina: int, pregunta: str):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 5, "filter": {"page": pagina}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Resultados en página {pagina}: {len(results)}")
    for r in results:
        print("\n--- CHUNK ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

# Ejemplo:
buscar_por_pagina(32, "alcance del suministro")
buscar_por_pagina(42, "caudal de la bomba")


🔎 Resultados en página 32: 5

--- CHUNK ---
facilidades para la conexión eléctrica deberán ser como mínimo de ½”, ¾” y 1” NPT. • Deberá ser suministrado con todos los accesorios para anclaje como placa base, pernos, arandelas y tuercas. También deberán ser incluidos en el suministro, como anillos u orejas de izaje. • El gabinete de control deberá tener aproximadamente las siguientes dimensiones: 250 mm (profundidad) x 600 mm (ancho) x 800 m
Metadatos: {'source': './data/FRDFMEW450. 1 (Especificacion Tecnica CIP POZO HUFF&PUFF).pdf', 'content_hash': 'c2b579693eb9d07689c1781c6d4221c0e601f8a1', 'prio': 'requisito', 'page': 32, 'chunk_id': '32_3', 'type': 'text'}

--- CHUNK ---
VICEPRESIDENCIA DE INGENIERÍA Y PROYECTOS CÓDIGO Elaborado Versión: 1 EDP-F-046 20/11/2017 INGENIERÍA DETALLADA POZO HUFF & PUFF FR UFE MR #: FRD/F-MEQ-450 REV: 1 LOCACIÓN FLOREÑA U Documento: FECHA: REQUISICIÓN DE MATERIALES Especialidad Mecánica PAQUETE DE INYECCIÓN DE 10/04/2024 QUÍMICOS 10. ANEXOS 1 10.1. LISTA 

In [28]:
# ============================================
# Paso 13.1 – Inventario de sistemas (Skids, TAGs y cantidades)
# ============================================

import json

def prueba_inventario_sistemas():
    pregunta = "¿Cuáles son los skids o sistemas de dosificación requeridos? Dame los TAG y las cantidades."
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 10, "filter": {"type": "table"}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Resultados prueba INVENTARIO (skids + tags + cantidades): {len(results)}")

    kv_count = 0
    for r in results:
        print("\n--- CHUNK INVENTARIO ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

        if "kv" in r.metadata:
            try:
                kv_dict = json.loads(r.metadata["kv"])
                print("KV dict:", kv_dict)
                kv_count += 1
            except Exception:
                print("KV (raw):", r.metadata["kv"])

    print(f"\n📊 Total de chunks con KV útiles: {kv_count}")

# Ejemplo de ejecución
prueba_inventario_sistemas()


🔎 Resultados prueba INVENTARIO (skids + tags + cantidades): 10

--- CHUNK INVENTARIO ---
Se realiza con el sistema de dosificación ensamblado.  | | | | | | | R | R | | | | | | | | | |   | | | | | | | R | | | | | | | | | | |   | | | | | | | R | R | | | | | | | | | |   | | | | | | | R | | | | | | | | | | |   | | | | | | | H | H | | | | | | | | | |
Metadatos: {'chunk_id': '21_3', 'prio': 'norma', 'content_hash': 'a982d80e34d34539ac4ca10b4526ff1bfff2a7f2', 'source': './data/FRDFMEQ450. 1 (Requisicion de Materiales CIP POZO HUFF&PUFF).pdf', 'type': 'table', 'page': 21}

--- CHUNK INVENTARIO ---
DEBE SUMINISTRAR, YA QUE SE TOMO UN CAUDAL MAXIMO DE INYECCION DE QUIMICO COMO EL CASO MAS CRITICO. | | |
Metadatos: {'source': './data/FRDFMEX450. 1 (Hoja de Datos CIP POZO HUFF&PUFF).pdf', 'chunk_id': '3_11', 'content_hash': 'eec843e714e48b85422c64115a85e8276904f7d8', 'page': 3, 'kv': '{"presion_psig": "50", "material_tanque": "304SS", "api_675": "true"}', 'prio': 'hoja_datos', 'type': 'table'}
KV 

In [29]:
# ============================================
# Paso 14 (v3) – Alcance + Datos de diseño de la bomba API 675 (robusto + actualizado)
# ============================================
import re, json
from langchain.schema import Document

def _j(x):
    try:
        return json.loads(x)
    except Exception:
        return {}

def _parse_nums(text: str):
    """Extrae caudal (GPH), presión (psig) y HP desde texto libre (fallback)."""
    caudal = None
    pres = None
    hp = None

    # Caudal "12 - 18 GPH" | "12–18 Gph" | "12 a 18 GPH" | simple "18 GPH"
    m = re.search(r'(\d+(?:[.,]\d+)?)\s*(?:-|–|a|to)\s*(\d+(?:[.,]\d+)?)\s*(gph|g\.?p\.?h\.?)\b', text, re.I)
    if m:
        caudal = f"{m.group(1)} - {m.group(2)}"
    else:
        m = re.search(r'\b(\d+(?:[.,]\d+)?)\s*(gph|g\.?p\.?h\.?)\b', text, re.I)
        if m:
            caudal = m.group(1)

    # Presión "@ 100 psi" | "100 psig"
    m = re.search(r'@?\s*(\d+(?:[.,]\d+)?)\s*(psi|psig)\b', text, re.I)
    if m:
        pres = m.group(1)

    # Potencia "1 HP"
    m = re.search(r'\b(\d+(?:[.,]\d+)?)\s*hp\b', text, re.I)
    if m:
        hp = m.group(1)

    return caudal, pres, hp

def prueba_alcance_y_datos_bomba_v3():
    q_alc = "alcance del suministro paquete de inyección de químicos"
    q_hd  = "bomba dosificadora API 675 caudal presión HP materiales GPH psig"

    # 1) Recuperar alcance
    retr_alc = vectorstore.as_retriever(
        search_kwargs={"k": 12, "filter": {"prio": "alcance"}},
        search_type="mmr"
    )
    alc_docs = retr_alc.get_relevant_documents(q_alc)

    if not alc_docs:  # fallback
        retr_alc_fb = vectorstore.as_retriever(search_kwargs={"k": 12}, search_type="mmr")
        alc_docs = retr_alc_fb.get_relevant_documents("alcance del suministro")

    # 2) Recuperar hojas de datos
    retr_hd = vectorstore.as_retriever(
        search_kwargs={"k": 20, "filter": {"prio": "hoja_datos"}},
        search_type="mmr"
    )
    hd_docs = retr_hd.get_relevant_documents(q_hd)

    # 3) Refuerzo directo desde la colección (KV + texto clave)
    coll = vectorstore._collection
    meta_hd = coll.get(where={"prio": "hoja_datos"}, include=["metadatas", "documents"])
    for md, doc in zip(meta_hd["metadatas"], meta_hd["documents"]):
        if not isinstance(md, dict):
            continue
        if ("kv" in md and any(k in md["kv"] for k in ["caudal_gph", "presion_psig", "hp_motor", "material_tanque", "material_mojado", "bombas_operativas", "bombas_respaldo"])) \
           or ("API 675" in (doc or "") or "GPH" in (doc or "") or "psig" in (doc or "")):
            hd_docs.append(Document(page_content=doc, metadata=md))

    # 4) De-duplicar por (page,type,primeros 60 chars)
    def _key(d): return (d.metadata.get("page"), d.metadata.get("type"), (d.page_content or "")[:60])
    seen = set()
    alc_unique, hd_unique = [], []
    for d in alc_docs:
        k = _key(d)
        if k not in seen:
            seen.add(k); alc_unique.append(d)
    for d in hd_docs:
        k = _key(d)
        if k not in seen:
            seen.add(k); hd_unique.append(d)

    # 5) Extraer KV + regex → síntesis
    datos = {
        "caudal_gph": None,
        "presion_psig": None,
        "hp_motor": None,
        "materiales": set(),
        "api_675": False,
        "bombas_operativas": None,
        "bombas_respaldo": None,
        "paginas_fuente": set(),
    }

    for d in hd_unique:
        md = d.metadata or {}
        if "page" in md:
            datos["paginas_fuente"].add(md["page"])

        # KV directo
        if "kv" in md:
            kv = _j(md["kv"])
            if not datos["caudal_gph"] and kv.get("caudal_gph"):
                datos["caudal_gph"] = kv["caudal_gph"]
            if not datos["presion_psig"] and kv.get("presion_psig"):
                datos["presion_psig"] = kv["presion_psig"]
            if not datos["hp_motor"] and kv.get("hp_motor"):
                datos["hp_motor"] = kv["hp_motor"]
            if kv.get("material_tanque"):
                datos["materiales"].add(kv["material_tanque"])
            if kv.get("material_mojado"):
                datos["materiales"].add(kv["material_mojado"])
            if str(kv.get("api_675")).lower() == "true":
                datos["api_675"] = True
            if kv.get("bombas_operativas"):
                datos["bombas_operativas"] = kv["bombas_operativas"]
            if kv.get("bombas_respaldo"):
                datos["bombas_respaldo"] = kv["bombas_respaldo"]

        # Fallback regex
        if any(v is None for v in (datos["caudal_gph"], datos["presion_psig"], datos["hp_motor"])):
            c, p, h = _parse_nums(d.page_content or "")
            if c and not datos["caudal_gph"]:
                datos["caudal_gph"] = c
            if p and not datos["presion_psig"]:
                datos["presion_psig"] = p
            if h and not datos["hp_motor"]:
                datos["hp_motor"] = h

    # 6) Mostrar resultados
    print(f"🔎 ALCANCE (k={len(alc_unique)}):")
    for r in alc_unique[:5]:
        print(f"\n— p.{r.metadata.get('page')} | {r.metadata.get('type')}")
        print((r.page_content or "")[:350].replace("\n", " "))

    print(f"\n🔎 HOJA DE DATOS (k={len(hd_unique)}):")
    for r in hd_unique[:6]:
        print(f"\n— p.{r.metadata.get('page')} | {r.metadata.get('type')}")
        print((r.page_content or "")[:350].replace("\n", " "))
        if "kv" in r.metadata:
            print("KV:", r.metadata["kv"])

    print("\n✅ EXTRACCIÓN SINTETIZADA:")
    print(f"  • Caudal (GPH): {datos['caudal_gph']}")
    print(f"  • Presión (psig): {datos['presion_psig']}")
    print(f"  • Potencia (HP): {datos['hp_motor']}")
    print(f"  • Materiales: {', '.join(sorted(datos['materiales'])) or '—'}")
    print(f"  • API 675: {'Sí' if datos['api_675'] else 'N/D'}")
    print(f"  • Bombas operativas: {datos['bombas_operativas'] or '—'}")
    print(f"  • Bombas respaldo: {datos['bombas_respaldo'] or '—'}")
    print(f"  • Páginas fuente (HD): {sorted(datos['paginas_fuente'])}")

    return {"alcance": alc_unique, "hoja_datos": hd_unique, "datos": datos}

# Ejecución:
prueba_alcance_y_datos_bomba_v3()



🔎 ALCANCE (k=12):

— p.1 | text
MAYORES Paquete de Inyección de Químicos

— p.5 | text
“Formato de solicitud técnicas”. 2. ALCANCE DEL SUMINISTRO 2.1. ALCANCE GENERAL A continuación, se listan el paquete objeto de esta requisición de materiales. El PROVEEDOR debe incluir los equipos, personal, herramientas, dispositivos, elementos y materiales necesarios y suficientes para que el PAQUETE sea funcional y opere adecuadamente bajo las c

— p.15 | text
PROJECT CONTRACTOR INGENIERÍA DETALLADA POZO HUFF & PUFF FR UFE LOCACIÓN FLOREÑA U CLIENT DOCUMENT NUMBER DATE FRD/F-MEW-450 10/04/2024 DOCUMENT TITLE Page 9 / 39 ESPECIFICACIÓN TÉCNICA PAQUETE DE INYECCIÓN DE Rev. 1 QUÍMICO POZO HUFF&PUFF FR UFE TAG I0-CIP-42001 7. Alcance del Suministro El PROVEEDOR será totalmente responsable del diseño y sumini

— p.15 | text
ejo, transporte, almacenaje y embalaje de estos. 1 7.1. Equipos y Componentes El PROVEEDOR deberá suministrar, probar y brindar asistencia técnica en la puesta en marcha del PAQUETE

{'alcance': [Document(id='81f26c9ba85910d9684790f8865bdd2636f8eef1', metadata={'type': 'text', 'content_hash': '81f26c9ba85910d9684790f8865bdd2636f8eef1', 'prio': 'alcance', 'chunk_id': '1_1', 'page': 1, 'source': './data/FRDFMEQ450. 1 (Requisicion de Materiales CIP POZO HUFF&PUFF).pdf'}, page_content='MAYORES\nPaquete de Inyección de Químicos'),
  Document(id='03d07d97e335b04d4564e35cd0f4564ff7ad85a7', metadata={'content_hash': '03d07d97e335b04d4564e35cd0f4564ff7ad85a7', 'type': 'text', 'chunk_id': '5_4', 'prio': 'alcance', 'source': './data/FRDFMEQ450. 1 (Requisicion de Materiales CIP POZO HUFF&PUFF).pdf', 'page': 5}, page_content='“Formato de solicitud técnicas”.\n2. ALCANCE DEL SUMINISTRO\n2.1. ALCANCE GENERAL\nA continuación, se listan el paquete objeto de esta requisición de materiales. El PROVEEDOR debe\nincluir los equipos, personal, herramientas, dispositivos, elementos y materiales necesarios y\nsuficientes para que el PAQUETE sea funcional y opere adecuadamente bajo las cond